#  1. Text preprocessing

No matter what kind of NLP you want to do - classify a text, extract named entities, disambiguate a word sense - you most curtainly need to do some preprocessing first. 

In this notebook we will go through basic preprocessing steps such as sentence segmentation, tokenization, lemmatization and look at one of more advanced approaches (bpe). We will also discuss existing text preprocessing tools for russian and english.

In [1]:
# you need to install this libraries to work with russian
# in Colab specifying pymystem3 version is important, you can drop it on your machine
# if pymorphy2[fast] does not install (it won't on windows) drop the [fast] part

In [124]:
!pip install pymystem3==0.1.10
!pip install pymorphy2[fast]

In [ ]:
# if you only want to work with english spacy is enough

In [ ]:
!pip install spacy

In [ ]:
# in case you don't have nltk

In [ ]:
!pip install nltk

In [2]:
# import everything
import string
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re, os, json
import spacy
nlp = spacy.load("en_core_web_sm")
mystem = Mystem()
morph = MorphAnalyzer()


## Cleaning the text

Textual data is often not just text. There can be tags, links, timestamps and other garbage that you'll probably want to get rid off. Regular expression is the tool that can help you with that.

**Re** is main python module for regular expressions. Look through the documentation if you never worked with it - https://docs.python.org/3/library/re.html
But don't try to read it all of it. The best way to learn it is practice. Let's get to it.

Here's some text. (It's in russian but for now it's ok that you don't understand it)

In [3]:
text = '''
Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!<br><br><img src="http://img172.imageshack.us/img172/8434/18274658kc4.png" alt="Сумасшедшее яйцо"><br><br><blockquote><h3>Что это такое?</h3><br>
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.<br>
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).<br><br><img src="https://habrastorage.org/getpro/habr/post_images/7c7/298/33c/7c729833cd942cc493e68833e3e0f12d.jpg" alt="Тепловое отслеживание популярности"><br></blockquote><br><br><a name="habracut"></a><br><br><blockquote><h3>Для кого это?</h3><br>
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн дает о себе знать — сервис массово популяризировался пол года-год назад. Относительно недорогой, но уж точно не из дешевых — сервис предполагает собой 4 платных линии и одну бесплатную. Дабы написать этот обзор я не поленился заплатить 19 долларов (в месяц) выбрав средний вариант — для нескольких проектов с включенными дополнительными функциями.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0b0/0a0/b16/0b00a0b16b1eda28e35f39487dcd2545.jpg" alt="Отслеживание ссылок"></blockquote><br><blockquote><img src="https://habrastorage.org/getpro/habr/post_images/0b5/433/892/0b54338921665ffb5a90930147296f5b.jpg" alt="Список по популярности"><br><br>
Да-да, сервис не бесплатен. Точнее бесплатная возможность потестировать есть, но она немного обрезана (можно отслеживать только 5000 посетителей и всего 4 страницы на сайте (внимание — 4 страницы, а не сайтов), т.е. вполне хватает для того, чтобы понять полезность сервиса).</blockquote><br><blockquote><h3>Зачем это?</h3><br>
С помощью этого сервиса можно тасовать блоки на сайте, которые полезны пользователям больше всего. Больше не нужно спорить создателям — какой блок где расположить. В этом им поможет <a href="http://ttp://www.crazyegg.com" title="Сумасшедшие яйца">Сумасшедшее яйцо</a>.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/38c/af8/e75/38caf8e753782a01dc6419d3902edd57.jpg" alt="Добавление проекта"></blockquote><br><blockquote>Также этот сервис поможет вам понять — в какой зоне сайта лучше всего располагать рекламу, когда в вашем сервисе речь зайдет о монетизации. Ведь альтруизм это хорошо, а деньги на содержание сервиса нужны, и не лишним будет вычислить зоны где реклама будет приносить наибольшую отдачу, и наименьшее раздражение у пользователей.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0dd/3fb/b34/0dd3fbb34709e77f3fbcd6523c7eac77.jpg" alt="еще полезности"></blockquote><br><blockquote><h3>Как это работает?</h3><br>
Никаких километровых скриптов вставлять не нужно, достаточно вставиь 2 строчки яваскрипта, и сервис начнет отслеживание. Насколько я понял — исполнительный скрипт работает на сервере <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>, поэтому ваш сайт от этого в производительности не потеряет ни секунды, а полезность довольно таки большая.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/a90/e4e/e67/a90e4ee6760978463e6306a2f5982e24.jpg" alt="управление популярностью"></blockquote><br><blockquote><h3>Сколько это стоит?</h3><br>
Как я уже упоминал — сервис далеко не бесплатен, хоть и имеет тестовую-бесплатную версию. Расскажу подробнее о тарификации.</blockquote><br><blockquote>1. Бесплатная тестовая версия. Включает в себя возможность отслеживания 5 000 посещений, 4 страницы на сайте.<br><br>
2. Базовая версия. В этой версии можно отследить 10 000 посещений, и 10 страниц, что вполне достаточно для среднего корпоративного сайта. Стоит базовый комплект — 9 долларов в месяц. В этот комплект включены все дополнительные функции.<br><br>
3. Версия «Стандарт». В неё входит возможность отслеживания 25 000 посещений на 20 страницах. Вполне подходит для тестирования нового стартапа. Стоит она 19 долларов в месяц, именно её я купил для тестирования сервиса, и написания этого обзора.<br><br>
4. Версия «Плюс». Отличается от предыдущей возможностью отслеживания 100 000 посещений, 50 страниц. Очень хороший тариф для крупных сервисов. Стоит 49 долларов в месяц. Довольно большие деньги за сервис, но они обычно с лихвой окупаются.<br><br>
5. Версия «Про». Стоит почти 100 долларов, имеет возможность отследить 250 000 посещений на ста страницах. Тариф подходит для монстров с большой посещаемостью и большим количеством страниц.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/f73/fb2/62d/f73fb262da4618a8dde67690cfd191ea.jpg" alt="Отслеживание статистики"></blockquote><br><blockquote><h3>Есть и аналоги</h3><br>
Я не поленился, и собрал еще пару ссылок с аналогами, которые предлагают такие же услуги, но немного дешевле.</blockquote><br><blockquote> 1. <a href="https://www.google.com/analytics/home/?hl=en" title="Шикарный сбор и анализ статистики">Google Analytics</a> — бесплатный сервис для сбора и анализа статистики, вывод статистики в наиболее наглядной форме, и без разнообразных рейтингов, счетчиков. Очень подробная и полезная вещь. Рекомендую, т.к. сам пользуюсь им для этого блога.<br><br>
2. <a href="http://www.mapsurface.com/" title="Сервис для отслеживания популярности блоков на сайте">MapSurface</a> — сам еще не использовал (т.к. предпочел CrazyEgg), но врядли будучи плохим сервис собрал бы множество положительных отзывов. К сожалению он сейчас находится в статусе закрытой беты.</blockquote><br><blockquote><h3>Вывод</h3><br>
Использовать можно, и нужно. Вот только тарифы довольно больно кусаются, но обычно эти деньги потом с лихвой отбиваются на повышении конвертации посетителей в деньги. Использовать сервис нужно для тестирования рекламных мест и удобства отдельных страниц, что помогает опять же повысить конвертацию. В общем и целом — <a href="http://ttp://www.crazyegg.com">полезная вещь</a> для каждого владельца сайтов, а для юзабилиста вообще практически обязательна. К счастью для людей, которые поиздержались деньгами в этом месяце — есть полезные аналоги.<br><br><img src="http://img241.imageshack.us/img241/2346/confettitnml5.jpg" alt="Интересная идея визуализации популярности - конфети"></blockquote><br><br>
Автор: <a href="http://www.birzool.com/" title="Я пишу о юзабилити веб интерфейсов">Ярослав Бирзул</a> (DezmASter).<br>
Источник: <a href="http://www.birzool.com/crazyegg/" title="Сумасшедшие яйца, первоисточник">Блог о юзабилити веб интерфейсов</a>.<br><br>
PS: Всех с прошедшим Новым годом! От всей души желаю вам всего самого-самого лучшего, чего вы желаете только в самых сокровенных мечтах. Удачно вам провести время.
'''

There are html tags in this text. Let's try to remove them.

Html tags are simple - they start with < and end with >.  We can write a simple regexp to match everything that is inside the tag.  
**re.sub** - is a function to replace what we match with something (or, in this case, nothing)  

In [4]:
def remove_tags_1(text):
    return re.sub(r'<[^>]+>', '', text)

Let's see how it works.

In [5]:
print(remove_tags_1(text))


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!Что это такое?
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).Для кого это?
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн да

It seems fine, but if we look closer we'll notice that there's no space between some sentences. Let's try to replace tags with space then.

In [6]:
def remove_tags_2(text):
    return re.sub(r'<[^>]+>', ' ', text)

In [7]:
print(remove_tags_2(text))


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя —  CrazyEgg . Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!       Что это такое?  
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов. 
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).             Для кого это?  
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный инт

Now there's too much spaces between sentences. But we can easily fix that with another regexp.

In [8]:
def remove_tags_3(text):
    no_tags_text = re.sub(r'<[^>]+>', ' ', text)
    no_space_sequences_text = re.sub('  +', ' ', no_tags_text) # replace all space sequences with only 1 space
    return no_space_sequences_text

In [9]:
print(remove_tags_3(text))


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg . Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично! Что это такое? 
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов. 
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые). Для кого это? 
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший диз

Now it's fine.

In [10]:
text = remove_tags_3(text)

## Sentence segmentation

Using regular expressions we can also split the text into sentences. However, the regular expression is a bit more complicated.

**re.split** - is a function to split text using a pattern.

Sentences usually end with .!? and there's space and uppercase letter after. So we can try something like:

In [11]:
re.split('[!?\.] [А-Я]', text)[:10]

['\nСегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg ',
 ' не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо',
 'апоминается',
 'тлично',
 'то это такое? \nКак уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов. \nСервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые)',
 'ля кого это? \nРазумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфей

The problem is that this regexp also removes the .!? and first uppercase letters!

We can solve it using __look ahead__ и __look behind__ (regex features).  
Here's the syntax:  
**(?<=pattern)** positive look-behind  
**(?<!pattern)** negative look-behind условие  
**(?=pattern)** positive look-ahead условие  
**(?!pattern)** negative look-ahead условие  

You can read about it here: https://www.regular-expressions.info/lookaround.html  
Or here (more general) - https://www.rexegg.com/regex-disambiguation.html

In short, Look behind and look ahead turn the pattern into a condition that should be met before of after. Everything that is inside the condition is not matched.

So let's wrap the left and the right part of out regexp and see what happens:

In [122]:
re.split('(?<=[\.?!]) +(?=[А-ЯЁ])', text.replace('\n', ' '))[:10]

[' Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?',
 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.',
 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).',
 'Для кого это?',
 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуи

Good sentences!

If you don't want to write something yourself, you can use **sent_tokenize** from *nltk* or **split_sentences** from *gensim*

In [12]:
sent_tokenize(text, 'russian')[:10]

['\nСегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?',
 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.',
 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).',
 'Для кого это?',
 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — инту

In [13]:
# it return a generator so we wrap it in a list
list(split_sentences(text))[:5]

['Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?']

## Tokenization

Now we may want to split our text into words. It sounds simple but in fact it's not. What we see in texts are not words, it's wordforms (for example, do, does, did are wordforms of one word). Besides, texts may contain other things (i.e. numbers, punctuation).  
We call such separate meaningful units of a text that are not words, nor necessarily wordforms **tokens**. And the process of splitting the text into tokens is called tokenization.


The simplest way to tokenize a text is __str.split__ method. It tokenizes a text on space sequences (1 or more)

In [14]:
text.split()[:20]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '—',
 'CrazyEgg',
 '.',
 'Я',
 'не',
 'знаю',
 'кому',
 'обязан',
 'сервис',
 'таким',
 'говорящим',
 'именем,']

NB: str.split(' ') is not the same as str.split(). 

But str.split() does not separate punctuation. Most of time, punctuation is not needed at all so we can throw it away using str.strip() function.


In [19]:
#common punctuation signs can be found in string.punctuation
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [20]:
# to work with russian we may quotation marks and three dots
string.punctuation += '«»—…“”'

In [119]:
[word.strip(string.punctuation) for word in text.split()][:10]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '',
 'CrazyEgg']

str.strip does not remove punctuation inside tokens

In [15]:
'как-нибудь'.strip(punctuation)

'как-нибудь'

In [16]:
"don't".strip(punctuation)

"don't"

Regular expressions can also help us tokenize the text:

In [24]:
# if we need punctuation
re.findall('\w+|\W', text)[:10]

['\n', 'Сегодняшняя', ' ', 'заметка', ' ', 'будет', ' ', 'о', ' ', 'сервисе']

In [25]:
# if we don't
re.findall('\w+', text)[:10]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 'CrazyEgg',
 'Я']

Nltk has ready-to-use tokenizers.

For example, **wordpunct_tokenizer** which uses this regexp - *'\w+|[^\w\s]+'* 

It can be used for russian and english because both languages use spaces. 

In [117]:
wordpunct_tokenize(text)[:10]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '—',
 'CrazyEgg']

In [23]:
wordpunct_tokenize("That's an example.")

['That', "'", 's', 'an', 'example', '.']

Another nltk tokenizer is **word_tokenize**. It also uses regular expressions but more sophisticated ones.
And they are crafted for english specifically. 

In [24]:
word_tokenize("That's an example.")

['That', "'s", 'an', 'example', '.']

Gensim also has tokenize function. 

In [26]:
list(tokenize("That's an example."))

['That', 's', 'an', 'example']

### Removing non-words

If we only need wordforms, we can use built-in **.isalpha()** and **len()** 

In [31]:
[word for word in word_tokenize(text) if word.isalpha() and len(word) < 30][:20]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 'CrazyEgg',
 'Я',
 'не',
 'знаю',
 'кому',
 'обязан',
 'сервис',
 'таким',
 'говорящим',
 'именем',
 'но',
 'оно']

str.isalpha() won't work on wordforms with - or ' inside

In [32]:
"don't".isalpha()

False

They can be splitted before filtering or you can use regular expression [a-z\\-'] instead if str.isalpha()

# Normalization

To group wordforms of one word together we need to do normalization. There are two ways of doing it: lemmatization or stemming.

## Stemming

Stemming is basically removing common endings (-s, -ed etc.) It is based on the assumption that words have roots (=stems) that do not change between wordforms.  

The most famous stemmer is Porter stemmer (or Snowball stemmer). 
You can read about it here - <https://medium.com/@eigenein/стеммер-портера-для-русского-языка-d41c38b2d340>  
Or here - <http://snowball.tartarus.org/algorithms/russian/stemmer.html>  

Here's a comment from Porter on why it is called Snoball:

`Since it effectively provides a ‘suffix STRIPPER GRAMmar’, I had toyed with the idea of calling it ‘strippergram’, but good sense has prevailed, and so it is ‘Snowball’ named as a tribute to SNOBOL, the excellent string handling language of Messrs Farber, Griswold, Poage and Polonsky from the 1960s.`

In [57]:
text = """The setting was a symposium between Buddhist monk-scholars and Western scientists in a Tibetan monastery in Southern India, fostering a dialogue in physics, biology, and brain science.

Buddhism has philosophical traditions reaching back to the fifth century B.C. It defines life as possessing heat (i.e., a metabolism) and sentience, that is, the ability to sense, to experience, and to act. According to its teachings, consciousness is accorded to all animals, large and small—human adults and fetuses, monkeys, dogs, fish, and even lowly cockroaches and mosquitoes. All of them can suffer; all their lives are precious.


"""

Snowball stemmer is part of the nltk:

In [58]:
from nltk.stem.snowball import SnowballStemmer

In [59]:
stemmer = SnowballStemmer('english')

In [60]:
[(word, stemmer.stem(word.strip(string.punctuation))) for word in text.split()][:300]

[('The', 'the'),
 ('setting', 'set'),
 ('was', 'was'),
 ('a', 'a'),
 ('symposium', 'symposium'),
 ('between', 'between'),
 ('Buddhist', 'buddhist'),
 ('monk-scholars', 'monk-scholar'),
 ('and', 'and'),
 ('Western', 'western'),
 ('scientists', 'scientist'),
 ('in', 'in'),
 ('a', 'a'),
 ('Tibetan', 'tibetan'),
 ('monastery', 'monasteri'),
 ('in', 'in'),
 ('Southern', 'southern'),
 ('India,', 'india'),
 ('fostering', 'foster'),
 ('a', 'a'),
 ('dialogue', 'dialogu'),
 ('in', 'in'),
 ('physics,', 'physic'),
 ('biology,', 'biolog'),
 ('and', 'and'),
 ('brain', 'brain'),
 ('science.', 'scienc'),
 ('Buddhism', 'buddhism'),
 ('has', 'has'),
 ('philosophical', 'philosoph'),
 ('traditions', 'tradit'),
 ('reaching', 'reach'),
 ('back', 'back'),
 ('to', 'to'),
 ('the', 'the'),
 ('fifth', 'fifth'),
 ('century', 'centuri'),
 ('B.C.', 'b.c'),
 ('It', 'it'),
 ('defines', 'defin'),
 ('life', 'life'),
 ('as', 'as'),
 ('possessing', 'possess'),
 ('heat', 'heat'),
 ('(i.e.,', 'i.e'),
 ('a', 'a'),
 ('metabo

In [61]:
stemmer.stem("ending")

'end'

In [62]:
stemmer.stem("goes")

'goe'

In [63]:
stemmer.stem("closed")

'close'

Obviously, the languages are more complex than just adding endings to stems. Suppletive forms are common both in english and russian. So lemmatization is preferred most of the time.

# Lemmatization

Lemmatization is replacing wordforms of one word with a standard form (lemma). For example, for verbs a standard form is an infinitive (but the choice of lemma is arbitrary, so it doesn't have to be an infinitive).   

Lemmatization requires a dictionary of all wordforms, for new words the lemma is inferred in a way similar to stemming.

For Russian Mystem and Pymorphy are best lemmatizers.


### Mystem


Mystem is a bit better and can tokenize

In [64]:
t = 'Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.'

In [65]:
mystem.lemmatize(t)[:10]

['сегодняшний', ' ', 'заметка', ' ', 'быть', ' ', 'о', ' ', 'сервис', ' ']

В целевом файле теперь лежит разобранный текст в jsonlines (json на каждой строчке)

### Pymorphy

Pymorphy only works with tokens

In [69]:
# основная функция - pymorphy.parse
[morph.parse(token)[0].normal_form for token in word_tokenize(t)]

['сегодняшний',
 'заметка',
 'быть',
 'о',
 'сервис',
 'отслеживание',
 'активность',
 'пользователь',
 '—',
 'crazyegg',
 '.',
 'я',
 'не',
 'знать',
 'кома',
 'обязать',
 'сервис',
 'такой',
 'говорящий',
 'имя',
 ',',
 'но',
 'оно',
 'работать',
 ',',
 'и',
 'хорошо',
 '.']

For English the best option is spacy. 

In [67]:
doc = nlp(text)

In [52]:
lemmas = []
for sent in doc.sents:
    for word in sent:
        lemmas.append(word.lemma_)

In [53]:
lemmas[:10]

['the',
 'setting',
 'be',
 'a',
 'symposium',
 'between',
 'Buddhist',
 'monk',
 '-',
 'scholar']

Spacy does sentence segmentation, tokenization, lemmatization and many other usefult things. 

## Removing stop-words

Often common words are not useful or can even worsen the quality of a model. Such words are called stop-words. The term comes from information retrieval (first mentioned by [Peter Luhn](https://en.wikipedia.org/wiki/Hans_Peter_Luhn) in 1959). Back in the day, stopwords were removed from the indecies to reduce memory consumption and improve search quality. After memory became cheap and IDF was invented people stopped doing that. But stopwords can still be useful in nlp.


In [55]:
# nltk has stopwords lists for different languages
stops = stopwords.words('english')
print(stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Subword tokenization

A lot of state-of-the-art nlp models use different tokenization methods. One of the most common algorithm is byte-pair encoding. It was first developed as a compression algorithm, only in 2016 it was succesfully applied to machine translation - https://www.aclweb.org/anthology/P16-1162

The main advantage of BPE is that it can be applied to any language. It doesn't assume that words are separated with spaces (or at all), it just learns tokenization rules directly from the text.

Also it is simple. Here's how it works:  
***
1) a given text is treated as a sequence of characters  
2) coocurrence statistics is collected  
3) most frequetly occuring characters are merged  
4) 2 and 3 steps are repeated N times
***

N is the only parameter of the algorith that has to be tuned. The mode N is, the longer character sequences will get.

The simplest implementation is the following:

In [106]:
import re, collections
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out



In [113]:
vocab = {'l o w </w>' : 5, 'l o w e r </w>' : 2,
         'n e w e s t </w>':6, 'w i d e s t </w>':3, 
         's t r a n g e r </w>':2, 'a n d </w>':10,
         's t a n d </w>':7
        }
num_merges = 5

for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        print(i)
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
print(vocab)

{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3, 'st r an g e r </w>': 2, 'and</w>': 10, 'st and</w>': 7}


Of course, it is not very efficient. If you need to perform BPE on large text you can use existing libraries like YouTokenToMe or SentencePiece.

Here's a medium post about YouTokenToMe - https://medium.com/@vktech/youtokentome-a-tool-for-quick-text-tokenization-from-the-vk-team-aa6341215c5a

Here's a github repo of SentencePiece - https://github.com/google/sentencepiece

When we get to neural networks, we'll try BPE on a real task and see how good it is.